# Get data from data warehouse

In [7]:
#I use librart sparklyr to connect and get data from Hadoop
library(sparklyr)
library(dplyr)

In [2]:
spark <- spark_connect(master='local')

In this code I allow the user to get a sample of data by month.
  + Month starts
  + Month ends
  
Depending on the date you crawl data in part1 how much data you will be able to get from datalake

In [3]:
month_start=as.integer(readline('Enter month start:'))
month_end=as.integer(readline('Enter month end:'))
df <- data.frame()
for (month in (month_start:month_end)){
    month<-sprintf("%02d", month)
    path=paste0("hdfs://localhost:9000/datalake/2023/",month,"/output_weather_in_*")
    print(path)
    if (month==sprintf("%02d",month_start)){
        df <- spark_read_parquet(spark,path=path)
        df <-df %>% mutate(Date=as.POSIXct(Date))
        df <- df %>% collect()
    }
    else{
        read <- spark_read_parquet(spark,path=path)
        read <-read %>% mutate(Date=as.POSIXct(Date))
        read <- read %>% collect()
        df <- rbind(df,read)
    }
    print(month)
}

Enter month start:6
Enter month end:7
[1] "hdfs://localhost:9000/datalake/2023/06/output_weather_in_*"
[1] "06"
[1] "hdfs://localhost:9000/datalake/2023/07/output_weather_in_*"
[1] "07"


I will try 2 months demo is 6 and 7

The default year here is 2023, I can completely tweak the date, but since this project is done in 2023 I will default it like that
- Path: "hdfs://localhost:9000/datalake/2023/",month,"/output_weather_in_*"

In [4]:
# I will rearrange by month to facilitate analysis and processing time seriese

In [8]:
df <- arrange(df, Date)

In [9]:
dim(df)

[1] 353  10

In [10]:
head(df)

Date,Chance of precipitation,Temperature,Feels like temperature °C,Wind gust mph,Visibility,Humidity.,UV .,Wind direction,Speed
<dttm>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>
2023-06-15 18:00:00,60,30,33,13,VG,73,1,WSW,7
2023-06-15 19:00:00,60,29,32,11,VG,80,0,WSW,6
2023-06-15 20:00:00,60,28,31,10,VG,82,0,WSW,6
2023-06-15 21:00:00,10,28,31,9,VG,84,0,WSW,5
2023-06-15 22:00:00,10,28,31,8,VG,86,0,SW,4
2023-06-15 23:00:00,10,27,31,8,VG,88,0,SW,4


In [11]:
tail(df)

Date,Chance of precipitation,Temperature,Feels like temperature °C,Wind gust mph,Visibility,Humidity.,UV .,Wind direction,Speed
<dttm>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>
2023-07-12 07:00:00,5,27,30,10,VG,90,2,WSW,5
2023-07-12 10:00:00,10,30,33,17,VG,73,8,W,8
2023-07-12 13:00:00,40,31,34,17,VG,65,8,W,8
2023-07-12 16:00:00,60,30,33,15,VG,72,2,WSW,8
2023-07-12 19:00:00,30,28,31,9,VG,83,0,WSW,5
2023-07-12 22:00:00,30,27,30,9,VG,88,0,SW,5


In [12]:
write.csv(df,'ouput_process.csv', row.names=FALSE)